This experiment, higher learning rate. Larger rollouts.

In [1]:
import deep_rl

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import torch
from torch.nn import functional as F
from torch.autograd import Variable
from torch import nn, optim
import torch.utils.data

# load as dask array
import time

import logging
import sys
import os
import glob
import numpy as np
import datetime
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [4]:
from deep_rl.utils.logger import get_logger, get_default_log_dir

from deep_rl.network.network_heads import CategoricalActorCriticNet, QuantileNet, OptionCriticNet, DeterministicActorCriticNet, GaussianActorCriticNet
from deep_rl.network.network_bodies import FCBody
from deep_rl.utils.normalizer import RunningStatsNormalizer, RescaleNormalizer
from deep_rl.component.task import ParallelizedTask

In [5]:
from world_models_sonic.models.vae import VAE5, loss_function_vae
from world_models_sonic.helpers.summarize import TorchSummarizeDf
from world_models_sonic.models.rnn import MDNRNN, MDNRNN2
from world_models_sonic.models.inverse_model import InverseModel
from world_models_sonic.models.world_model import WorldModel
from world_models_sonic.custom_envs.env import make_env
from world_models_sonic.custom_envs.wrappers import RandomGameReset
from world_models_sonic import config
from world_models_sonic.helpers.deep_rl import PPOAgent, run_iterations, SonicWorldModelDeepRL, CategoricalWorldActorCriticNet, Config

Importing 0 potential games...
Imported 0 games


# Init

In [7]:
cuda = torch.cuda.is_available()
env_name = 'sonic256'
z_dim = 512  # latent dimensions
channels = 3*4

# RNN
action_dim = 10
image_size = 128

verbose = True  # Set this true to render (and make it go slower)

NAME = 'RNN_v3b_128im_512z_1512_v6k_VAE5_all'
ppo_save_file = './outputs/{NAME}/PPO_512z_all_g.pkl'.format(NAME=NAME)

if not os.path.isdir('./outputs/{NAME}'.format(NAME=NAME)):
    os.makedirs('./outputs/{NAME}'.format(NAME=NAME))

# Log to file and stream
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(NAME)

log_dir = log_dir='./outputs/{NAME}'.format(NAME=NAME)
print(log_dir)

deep_rl_logger = get_logger(
    NAME,
    file_name='deep_rl_ppo.log',
    level=logging.INFO,
    log_dir='./outputs/{NAME}'.format(NAME=NAME), )

./outputs/RNN_v3b_128im_512z_1512_v6k_VAE5_all


# World model

In [8]:
# Load VAE
# TODO swap z and k dim, since it's inconsistent with other models
vae = VAE5(image_size=image_size, z_dim=128, conv_dim=64, code_dim=8, k_dim=z_dim, channels=channels)
    
# Load MDRNN
action_dim, hidden_size, n_mixture, temp = action_dim, z_dim*2, 5, 0.0

mdnrnn = MDNRNN(z_dim, action_dim, hidden_size, n_mixture, temp)
    
finv = InverseModel(z_dim, action_dim, hidden_size=z_dim*2)
    
world_model = WorldModel(vae, mdnrnn, finv, logger=deep_rl_logger, lambda_vae_kld=1 / 1024., lambda_finv=1/100, lambda_vae=1/64, lambda_loss=1000)
world_model = world_model.train()
if cuda:
    world_model = world_model.cuda()

In [9]:
# Optimiser for world models
import torch.optim.lr_scheduler
torch.cuda.empty_cache()
optimizer = optim.Adam(world_model.parameters(), lr=3e-5)

world_model.optimizer = optimizer

# Train

In [10]:
z_state_dim=world_model.mdnrnn.z_dim + world_model.mdnrnn.hidden_size  + world_model.mdnrnn.action_dim


def task_fn(log_dir):
    return SonicWorldModelDeepRL(
        env_fn=lambda: RandomGameReset(make_env(
            'sonic', max_episode_steps=1000, to_gray=False, image_size=image_size)),
        log_dir=log_dir,
        verbose=verbose
    )

config = Config()

verbose = False  # Set this true to render (and make it go slower)
config.num_workers = 1 if verbose else 5
config.task_fn = lambda: ParallelizedTask(
    task_fn, config.num_workers, single_process=config.num_workers == 1)
config.optimizer_fn = lambda params: torch.optim.Adam(params, 3e-4)
config.network_fn = lambda state_dim, action_dim: CategoricalWorldActorCriticNet(
    state_dim, action_dim, FCBody(z_state_dim, hidden_units=(64, 64), gate=F.relu), gpu=0 if cuda else -1, world_model_fn=lambda: world_model,
    render=(config.num_workers==1 and verbose),
    z_shape=(32, 16)
)
config.discount = 0.99
config.logger = deep_rl_logger
config.use_gae = True
config.gae_tau = 0.95
config.entropy_weight = 0.001
config.gradient_clip = 0.4
config.rollout_length = 1*24
config.optimization_epochs = 10
config.num_mini_batches = 8*1
config.ppo_ratio_clip = 0.2
config.iteration_log_interval = 10

config.train_world_model = True
config.world_model_batch_size = 1

# I tuned these so the intrinsic reward was 1) within an order of magnitude of the extrinsic. 2) smaller, 3) negative when stuck
# TODO use reward normalisers etc to reduce the need for these hyperparameters
config.curiosity = True
config.curiosity_only = True
config.curiosity_weight = 0.1
config.curiosity_boredom = 0 # how many standard deviations above the mean does it's new experience need to be, so it's not bored
config.intrinsic_reward_normalizer = RescaleNormalizer()
config.reward_normalizer = RunningStatsNormalizer()
agent = PPOAgent(config)

print('rollout of ', config.rollout_length*config.num_workers)
print('mini batch', (config.rollout_length*config.num_workers)/config.num_mini_batches)
print('sequence of batch', (config.rollout_length))

if os.path.isfile(ppo_save_file):
    print('loading ppo_save_file', ppo_save_file, 'modified', time.ctime(os.path.getmtime(ppo_save_file)))
    agent.load(ppo_save_file)
    
    # also load normalizers
    state_dict = torch.load(ppo_save_file.replace('.pkl', '-intrinsic_reward_normalizer.pkl'))
    config.intrinsic_reward_normalizer.load_state_dict(state_dict)

    state_dict = torch.load(ppo_save_file.replace('.pkl', '-reward_normalizer.pkl'))
    config.reward_normalizer.load_state_dict(state_dict)
else:
    print("couldn't find save")

game: SonicAndKnuckles3-Genesis state: HiddenPalaceZone
game: SonicTheHedgehog2-Genesis state: MysticCaveZone.Act1
game: SonicTheHedgehog2-Genesis state: MysticCaveZone.Act2
game: SonicTheHedgehog-Genesis state: MarbleZone.Act3
game: SonicAndKnuckles3-Genesis state: CarnivalNightZone.Act1
rollout of  120
mini batch 15.0
sequence of batch 24
loading ppo_save_file ./outputs/RNN_v3b_128im_512z_1512_v6k_VAE5_all/PPO_512z_all_g.pkl modified Wed Jun  6 15:30:08 2018


In [ ]:
try:
    run_iterations(agent, log_dir=log_dir)
except:
    if config.num_workers == 1:
        agent.task.tasks[0].env.close()
    else:
        [t.close() for t in agent.task.tasks]
    print("saving", ppo_save_file)
    agent.save(ppo_save_file)
    torch.save(config.intrinsic_reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-intrinsic_reward_normalizer.pkl'))
    torch.save(config.reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-reward_normalizer.pkl'))

    # Backup since it sometimes get's corrupted
    ts = datetime.datetime.utcnow().strftime('%Y%m%d_%H-%M-%S')
    print("saving backup",
          ppo_save_file.replace('.pkl', '-%s.pkl' % ts),)
    agent.save(ppo_save_file.replace('.pkl', '-%s.pkl' % ts))
    raise

rollout extrinsic, intrinsic reward [min/mean/max]: -0.3935/-0.3935/-0.3935, 0.0113/0.0113/0.0113


2018-06-06 15:30:30,638 - RNN_v3b_128im_512z_1512_v6k_VAE5_all - INFO: loss_rnn=35.5682, loss_inv= 7.3849=0.0100 * 738.4918, loss_vae=1.8564=0.0156 * (118.5369 + 0.0010 * 280.0177)
2018-06-06 15:30:30,639 - RNN_v3b_128im_512z_1512_v6k_VAE5_all - INFO: total steps 120, min/mean/max reward 0.0000/0.0000/0.0000 of 5
2018-06-06 15:30:30,640 - RNN_v3b_128im_512z_1512_v6k_VAE5_all - INFO: running min/mean/max reward 0.0000/0.0000/0.0000 of 5 26.2474 step/s


rollout extrinsic, intrinsic reward [min/mean/max]: -0.2834/-0.2834/-0.2834, 0.0134/0.0134/0.0134
rollout extrinsic, intrinsic reward [min/mean/max]: -0.3792/-0.3792/-0.3792, 0.0156/0.0156/0.0156
rollout extrinsic, intrinsic reward [min/mean/max]: -0.3538/-0.3538/-0.3538, 0.0163/0.0163/0.0163
rollout extrinsic, intrinsic reward [min/mean/max]: -0.1829/-0.1829/-0.1829, 0.0126/0.0126/0.0126
rollout extrinsic, intrinsic reward [min/mean/max]: -0.0617/-0.0617/-0.0617, 0.0131/0.0131/0.0131
rollout extrinsic, intrinsic reward [min/mean/max]: -0.1859/-0.1859/-0.1859, 0.0141/0.0141/0.0141
rollout extrinsic, intrinsic reward [min/mean/max]: -0.4524/-0.4524/-0.4524, 0.0124/0.0124/0.0124
rollout extrinsic, intrinsic reward [min/mean/max]: -0.4534/-0.4534/-0.4534, 0.0135/0.0135/0.0135
rollout extrinsic, intrinsic reward [min/mean/max]: -0.4505/-0.4505/-0.4505, 0.0116/0.0116/0.0116
rollout extrinsic, intrinsic reward [min/mean/max]: -0.4436/-0.4436/-0.4436, 0.0128/0.0128/0.0128


2018-06-06 15:31:15,486 - RNN_v3b_128im_512z_1512_v6k_VAE5_all - INFO: loss_rnn=35.5181, loss_inv= 7.4683=0.0100 * 746.8251, loss_vae=2.0685=0.0156 * (132.1312 + 0.0010 * 258.3806)
2018-06-06 15:31:15,487 - RNN_v3b_128im_512z_1512_v6k_VAE5_all - INFO: total steps 1320, min/mean/max reward 0.0000/0.4697/2.3487 of 5
2018-06-06 15:31:15,488 - RNN_v3b_128im_512z_1512_v6k_VAE5_all - INFO: running min/mean/max reward 0.0000/0.0427/2.3487 of 55 26.7826 step/s


rollout extrinsic, intrinsic reward [min/mean/max]: -0.4380/-0.4380/-0.4380, 0.0146/0.0146/0.0146
rollout extrinsic, intrinsic reward [min/mean/max]: -0.4299/-0.4299/-0.4299, 0.0123/0.0123/0.0123
rollout extrinsic, intrinsic reward [min/mean/max]: -0.3482/-0.3482/-0.3482, 0.0120/0.0120/0.0120
rollout extrinsic, intrinsic reward [min/mean/max]: -0.0520/-0.0520/-0.0520, 0.0120/0.0120/0.0120
rollout extrinsic, intrinsic reward [min/mean/max]: -0.2113/-0.2113/-0.2113, 0.0115/0.0115/0.0115
rollout extrinsic, intrinsic reward [min/mean/max]: -0.2839/-0.2839/-0.2839, 0.0129/0.0129/0.0129
rollout extrinsic, intrinsic reward [min/mean/max]: -0.4063/-0.4063/-0.4063, 0.0111/0.0111/0.0111
rollout extrinsic, intrinsic reward [min/mean/max]: -0.3954/-0.3954/-0.3954, 0.0117/0.0117/0.0117
rollout extrinsic, intrinsic reward [min/mean/max]: -0.2320/-0.2320/-0.2320, 0.0117/0.0117/0.0117


In [ ]:
agent.save(ppo_save_file)
torch.save(config.intrinsic_reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-intrinsic_reward_normalizer.pkl'))
torch.save(config.reward_normalizer.state_dict(), ppo_save_file.replace('.pkl', '-reward_normalizer.pkl'))


To monitor with tensorboard
```sh
cd ~/Documents/projects/retro_sonic_comp/world-models-pytorch/log 
tensorboard  --logdir .
#then open http://localhost:6006/#scalars
```


In [ ]:
agent.save(ppo_save_file)
ppo_save_file

# summarize

In [ ]:
agent.network.network

In [ ]:
from IPython.display import display

with torch.no_grad():
    img = np.random.randn(image_size, image_size, 3)
    action = np.array(np.random.randint(0,action_dim))[np.newaxis]
    action = Variable(torch.from_numpy(action)).float().cuda()[np.newaxis]
    gpu_img = Variable(torch.from_numpy(img[np.newaxis].transpose(0, 3, 1, 2))).float().cuda()
    if cuda:
        gpu_img = gpu_img.cuda()
    with TorchSummarizeDf(vae) as tdf:
        x, mu_vae, logvar_vae = vae.forward(gpu_img)
        z = vae.sample(mu_vae, logvar_vae)
        df_vae = tdf.make_df()

    display(df_vae[df_vae.level<2])
    
    with TorchSummarizeDf(mdnrnn) as tdf: 
        pi, mu, sigma, hidden_state = mdnrnn.forward(z.unsqueeze(1).repeat((1,2,1)))
        z_next = mdnrnn.sample(pi, mu, sigma)
        df_mdnrnn = tdf.make_df()
    
    display(df_mdnrnn)
    

    with TorchSummarizeDf(finv) as tdf:
        finv(z.repeat((1,2,1)), z_next)   
        df_finv = tdf.make_df()
    display(df_finv)

    with TorchSummarizeDf(world_model) as tdf:
        world_model(gpu_img, action)
        df_world_model = tdf.make_df()
    display(df_world_model[df_world_model.level<2])
    
    del img, action, gpu_img, x, mu, z, z_next, mu_vae, pi, sigma, logvar_vae